In [25]:
import pandas as pd
import numpy as np
import collections
import pickle

In [26]:
data = pd.read_csv("./data/RPL.csv", encoding = 'cp1251', delimiter=';')
data.head()

,Часть,Год,Команда,Соперник,Минуты,Схема,Забито,Пропущено,Удары,Удары в створ,Передачи,Точные передачи,Навесы,Точные навесы,Владение,xG,PPDA,Победитель,Проигравший
0,1,2018,Ахмат,Арсенал,95,4-2-3-1 (100.0%),2,0,19,8,422,361,10,3,52.64,3.19,6.36,Ахмат,Арсенал
1,1,2018,Ахмат,Енисей,94,3-5-2 (100.0%),1,1,7,2,347,247,10,2,37.84,0.19,19.44,Ничья,Ничья
2,1,2018,Ахмат,ЦСКА,96,5-3-2 (79.58%),0,2,11,3,375,291,9,1,46.41,1.02,9.97,ЦСКА,Ахмат
3,1,2018,Ахмат,КС,94,5-3-2 (100.0%),2,1,8,2,455,372,14,6,52.39,0.95,9.19,Ахмат,КС
4,1,2018,Ахмат,Зенит,94,5-3-2 (93.08%),0,1,11,3,440,357,16,4,48.79,0.62,15.52,Зенит,Ахмат


In [27]:
RPL_2018_2019 = pd.read_csv('./data/teams 20-21.csv', encoding = 'cp1251')

teamList = RPL_2018_2019['Team Name'].tolist()
teamList

['Ахмат',
 'Зенит',
 'Краснодар',
 'Локомотив',
 'Ростов',
 'Рубин',
 'Спартак',
 'Урал',
 'Уфа',
 'ЦСКА']

Удаляем матчи с командами, которые не участвуют в сезоне 2020/2021:

In [28]:
deleteTeam = [x for x in pd.unique(data['Соперник']) if x not in teamList]
for name in deleteTeam:
    data = data[data['Команда'] != name]
    data = data[data['Соперник'] != name]
data = data.reset_index(drop=True)
print(data)

     Часть   Год Команда   Соперник  Минуты             Схема  Забито  \
0        1  2018   Ахмат       ЦСКА      96    5-3-2 (79.58%)       0   
1        1  2018   Ахмат      Зенит      94    5-3-2 (93.08%)       0   
2        1  2018   Ахмат  Краснодар      97    5-3-2 (100.0%)       1   
3        1  2018   Ахмат  Локомотив      94    3-5-2 (100.0%)       0   
4        1  2018   Ахмат    Спартак      98    5-3-2 (100.0%)       2   
5        1  2018   Ахмат        Уфа      95  4-2-3-1 (93.18%)       2   
6        1  2018   Ахмат       Урал      95  4-1-4-1 (100.0%)       1   
7        1  2018   Ахмат      Рубин      96  4-2-3-1 (82.94%)       1   
8        1  2018   Ахмат     Ростов      94  4-2-3-1 (31.02%)       0   
9        2  2018   Ахмат       Урал      97  4-2-3-1 (94.18%)       0   
10       2  2018   Ахмат    Спартак      94    4-5-1 (100.0%)       3   
11       2  2018   Ахмат  Локомотив      96  4-4-1-1 (100.0%)       0   
12       2  2018   Ахмат      Рубин      95    5-3-

In [29]:
#Функция, возвращающая статистику команды за сезон 
def GetSeasonTeamStat(team, season):
    goalScored = 0 #Голов забито
    goalAllowed = 0 #Голов пропущено

    gameWin = 0 #Выиграно
    gameDraw = 0 #Ничья
    gameLost = 0 #Проиграно

    totalScore = 0 #Количество набранных очков

    matches = 0 #Количество сыгранных матчей
    
    xG = 0 #Ожидаемые голы
    
    shot = 0 #Удары
    shotOnTarget = 0 #Удары в створ
    
    cross = 0 #Навесы
    accurateCross = 0 #Точные навесы
    
    totalHandle = 0 #Владение мячом
    averageHandle = 0 #Среднее владение мячом за матч
    
    Pass = 0 #Пасы
    accuratePass = 0 #Точные пасы
    
    PPDA = 0 #Интенсивность прессинга в матче

    for i in range(len(data)):
        if (((data['Год'][i] == season) and (data['Команда'][i] == team) and (data['Часть'][i] == 2)) or ((data['Год'][i] == season-1) and (data['Команда'][i] == team) and (data['Часть'][i] == 1))):
            matches += 1
                
            goalScored += data['Забито'][i]
            goalAllowed += data['Пропущено'][i]

            if (data['Забито'][i] > data['Пропущено'][i]):
                totalScore += 3
                gameWin += 1
            elif (data['Забито'][i] < data['Пропущено'][i]):
                gameLost +=1
            else:
                totalScore += 1
                gameDraw += 1
            
            xG += data['xG'][i]
            
            shot += data['Удары'][i]
            shotOnTarget += data['Удары в створ'][i]
            
            Pass += data['Передачи'][i]
            accuratePass += data['Точные передачи'][i]
            
            totalHandle += data['Владение'][i]
            
            cross += data['Навесы'][i]
            accurateCross += data['Точные навесы'][i]
            
            PPDA += data['PPDA'][i]

    averageHandle = round(totalHandle/matches, 3) #Владение мячом в среднем за матч
    
    return [gameWin, gameDraw, gameLost, 
            goalScored, goalAllowed, totalScore, 
            round(xG, 3), round(PPDA, 3),
            shot, shotOnTarget, 
            Pass, accuratePass,
            cross, accurateCross,
            round(averageHandle, 3)]

In [30]:
GetSeasonTeamStat("Спартак", 2018) #Статистика Спартака за сезон 2017/2018 

[10, 3, 5, 33, 23, 33, 29.62, 162.64, 280, 105, 7981, 6534, 262, 102, 52.473]

In [31]:
returnNames = ["Выиграно", "Ничья", "Проиграно",
               "\nГолов забито", "Голов пропущено", "\nНабрано очков",
               "\nxG (за сезон)", "PPDA (за сезон)",
               "\nУдары", "Удары в створ", 
               "\nПасы", "Точные пасы",
               "\nНавесы", "Точные навесы",
                "\nВладение (в среднем за матч)"]

for i, n in zip(returnNames, GetSeasonTeamStat("Спартак", 2018)):
        print(i, n)

Выиграно 10
Ничья 3
Проиграно 5

Голов забито 33
Голов пропущено 23

Набрано очков 33

xG (за сезон) 29.62
PPDA (за сезон) 162.64

Удары 280
Удары в створ 105

Пасы 7981
Точные пасы 6534

Навесы 262
Точные навесы 102

Владение (в среднем за матч) 52.473


In [32]:
#Функция, возвращающая статистику всех команд за сезон
def GetSeasonAllTeamStat(season):
    annual = collections.defaultdict(list)
    for team in teamList:
        team_vector = GetSeasonTeamStat(team, season)
        annual[team] = team_vector
    return annual

Функция, которая будет возвращать обучающие данные. Она создает словарь с векторами команд за все сезоны. Для каждой игры функция рассчитывает разницу между векторами команд за определенный сезон и записывает в xTrain. Затем функция присваивает yTrain значение 1, если команда хозяев выигрывает, и 0 в противном случае.

In [33]:
#Получение тренировычных данных
def GetTrainingData(seasons):
    totalNumGames = 0
    for season in seasons:
        annual = data[data['Год'] == season]
        totalNumGames += len(annual.index)
    numFeatures = len(GetSeasonTeamStat('Зенит', 2016)) #случайная команда для определения размерности
    xTrain = np.zeros(( totalNumGames, numFeatures))
    yTrain = np.zeros(( totalNumGames ))
    indexCounter = 0
    for season in seasons:
        team_vectors = GetSeasonAllTeamStat(season)
        annual = data[data['Год'] == season]
        numGamesInYear = len(annual.index)
        xTrainAnnual = np.zeros(( numGamesInYear, numFeatures))
        yTrainAnnual = np.zeros(( numGamesInYear ))
        counter = 0
        for index, row in annual.iterrows():
            team = row['Команда']
            t_vector = team_vectors[team]
            rivals = row['Соперник']
            r_vector = team_vectors[rivals]
            
            diff = [a - b for a, b in zip(t_vector, r_vector)]
            
            if len(diff) != 0:
                xTrainAnnual[counter] = diff
            if team == row['Победитель']:
                yTrainAnnual[counter] = 1
            else: 
                yTrainAnnual[counter] = 0
            counter += 1   
        xTrain[indexCounter:numGamesInYear+indexCounter] = xTrainAnnual
        yTrain[indexCounter:numGamesInYear+indexCounter] = yTrainAnnual
        indexCounter += numGamesInYear
    return xTrain, yTrain

Поучаем обучающие данные за все сезоны с 2015/2016 по 2018/2019:

In [34]:
years = range(2016,2018)
xTrain, yTrain = GetTrainingData(years)

Для прогнозирования вероятности выигрыша будем использовать алгоритм машинного обучения LinearRegression из библиотеки Scikit-Learn

In [35]:
from sklearn.linear_model import LinearRegression

In [36]:
model = LinearRegression()
model.fit(xTrain, yTrain)
pickle.dump(model, open("linear_regression_model.pkl", "wb"))

Функция, которая будет возвращать прогнозы. Она будет возвращать значение в промежутке от 0 до 1, где 0 — это проигрыш, а 1 — это выигрыш.

In [37]:
def createGamePrediction(team1_vector, team2_vector):
    diff = [[a - b for a, b in zip(team1_vector, team2_vector)]]
    predictions = model.predict(diff)
    return predictions

In [38]:
team1_name = "Зенит"
team2_name = "Спартак"

team1_vector = GetSeasonTeamStat(team1_name, 2019)
team2_vector = GetSeasonTeamStat(team2_name, 2019)

print ('Вероятность, что выиграет ' + team1_name + ':', createGamePrediction(team1_vector, team2_vector))
print ('Вероятность, что выиграет ' + team2_name + ':', createGamePrediction(team2_vector, team1_vector))

Вероятность, что выиграет Зенит: [0.43484215]
Вероятность, что выиграет Спартак: [0.32451614]


In [39]:
for team_name in teamList:
    team1_name = "ЦСКА"
    team2_name = team_name
    
    if(team1_name != team2_name):
        team1_vector = GetSeasonTeamStat(team1_name, 2019)
        team2_vector = GetSeasonTeamStat(team2_name, 2019)

        print(team1_name, createGamePrediction(team1_vector, team2_vector), " - ", team2_name, createGamePrediction(team2_vector, team1_vector,))

ЦСКА [0.41724751]  -  Ахмат [0.34211078]
ЦСКА [0.3395877]  -  Зенит [0.41977059]
ЦСКА [0.41313067]  -  Краснодар [0.34622761]
ЦСКА [0.43639616]  -  Локомотив [0.32296213]
ЦСКА [0.35061166]  -  Ростов [0.40874663]
ЦСКА [0.35584154]  -  Рубин [0.40351675]
ЦСКА [0.39475071]  -  Спартак [0.36460758]
ЦСКА [0.53132149]  -  Урал [0.2280368]
ЦСКА [0.41090016]  -  Уфа [0.34845812]
